<a href="https://colab.research.google.com/github/bubuloMallone/NeuralNetworksEX/blob/main/mlp_internals.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>